In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sklearn 
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
train= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
test["playtime_forever"]=0
data = pd.concat([train,test],axis=0, sort=False)
data.fillna(0, inplace=True)

In [3]:
del data['id']
del data['is_free']


dummy1 = data["genres"].str.get_dummies(",")
del data["genres"]

dummy2 = data["categories"].str.get_dummies(",")
del data["categories"]

dummy3=data["tags"].str.get_dummies(",")
test_dummy3=test["tags"].str.get_dummies(",")
del data["tags"]

for var in dummy1.columns:
    if var in dummy2.columns:
        del dummy1[var]
    if var in dummy3.columns:
        del dummy1[var]
for var in dummy2.columns:
    if var in dummy3.columns:
        del dummy2[var]

In [4]:
# operate datetime variable
a=pd.to_datetime(data["purchase_date"])
b=pd.datetime(2019,11,1)-a
#b=a-pd.datetime(2009,11,1)
data["purchase_date"]=b.astype('timedelta64[D]')
#A=pd.to_datetime(test_merge["purchase_date"])
#B=pd.datetime(2019,11,1)-A
#test_merge["purchase_date"]=B.astype('timedelta64[D]')

a=pd.to_datetime(data["release_date"])
b=pd.datetime(2019,11,1)-a
#b=a-pd.datetime(2009,11,1)
data["release_date"]=b.astype('timedelta64[D]')
#A=pd.to_datetime(test_merge["release_date"])
#B=pd.datetime(2019,11,1)-A
#test_merge["release_date"]=b.astype('timedelta64[D]')
np.where(np.isnan(data))

(array([], dtype=int64), array([], dtype=int64))

In [5]:
y=data["playtime_forever"].values
del data["playtime_forever"]

In [6]:
array_data=data.values
array_tag=dummy1.values
array_tag2=dummy2.values
array_tag3=dummy3.values
x=np.hstack([array_data,array_tag,array_tag2,array_tag3])
x_train,x_test=x[:357],x[357:]
y_train,y_test=y[:357],y[357:]
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [7]:
#x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size=0.9)
std_x = MinMaxScaler() 
x_train = std_x.fit_transform(x_train)
#x_val = std_x.transform(x_val)
x_test = std_x.transform(x_test)
 
std_y  = MinMaxScaler() 
y_train = std_y.fit_transform(y_train)
#y_val = std_y.transform(y_val)

In [8]:
rd1= Ridge(alpha=0.3)
 
rd1.fit(x_train,y_train)
 
y_rd_predict2 = rd1.predict(x_train)
 
y_rd_predict2 = std_y.inverse_transform(y_rd_predict2)
y_rd_predict2=(y_rd_predict2>0)*y_rd_predict2
print("均方误差为：",metrics.mean_squared_error(y_rd_predict2,std_y.inverse_transform(y_train)))

均方误差为： 14.457051118972666


In [9]:
y_predict2 = rd1.predict(x_test)
 
y_predict2 = std_y.inverse_transform(y_predict2)
y_predict2=(y_predict2>0)*y_predict2
b=np.arange(90).reshape(90,1)
c=np.c_[b,y_predict2]
np.savetxt('predict.csv', c, delimiter = ',')